In [49]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# Some simulation codes

First, retrieve all scene objects' configuration and position. 

In [50]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [51]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

listazinha = get_scene_objects_info(client_id, scene_objects)

for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

# Testing some random ideas and useful functions

In [54]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [55]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


In [73]:
# re orient the robot

handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,1.57)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [57]:
sim.simxGetObjectOrientation(client_id, handle_dr20,-1, sim.simx_opmode_blocking) # see if orientation is ok

(0, [0.001957559259608388, 2.778682392090559e-05, 1.5699542760849])

In [58]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14 # robot orientation in deg

90.0030849845546

In [59]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [60]:
for element in info_list:
    normals = get_normals_orientation(element['object_orientation'][2])
    # position_x = round
    print(f'{element["object_name"]}: {normals}')

Cuboid_0: (4.856742659257329e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_1: (-1.4431441298492125e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_2: (2.8223240372909244e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_3: (5.502494439336651e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_4: (-1.4694823877335134e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)


In [61]:
x_min, y_min, z_min, x_max, y_max, z_max = get_bounding_box_corners_positions(client_id,listazinha[0]['object_handler'],"model")

In [71]:
# in order to test the map_local_coordinates_to_global_coordinates() power to map the corners local coordinates to global values,
# let's use the cuboid 0. It is global corners must be something like
# [-1.5, 0.5] [-2, 0.5] [-2, 0] [-1.5, 0]

cuboid_number = 0
local_c_cuboid = get_bounding_box_corners_local_coordinates(client_id, listazinha[cuboid_number]['object_handler'])
orientation_cuboid = listazinha[cuboid_number]['object_orientation']
orientation_cuboid.reverse() # notice that the orientation vector is reversed (theta is naturally the last, but we need to have it as the first value)
position_cuboid = listazinha[cuboid_number]['object_position']
global_coordinates_cuboid = map_local_coordinates_to_global_coordinates(local_c_cuboid, orientation_cuboid, position_cuboid)

we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
- Get the bounding box vertices from $x_{min}, x_{max}, y_{min}, y_{max}, z_{min}, z_{max}$
- Map bounding box vertices (in object's reference frame to global reference frame)
- Test applicability using the normal's orientation (i think i'll need to check sine and cossine, or limit the orientation within [0,2pi] )
- Create Configuration Vertice 

In [63]:
cuboid_number = 0
orient_cuboid = info_list[cuboid_number]['object_orientation']
n_4_obj, n_1_obj, n_2_obj, n_3_obj = get_normals_orientation(orient_cuboid[2]) 
# notice the numering of the normal vectors

In [64]:
pi = 3.14159265359
orient_robot = robot_information['object_orientation']
n_4_robot, n_1_robot, n_2_robot, n_3_robot = get_normals_orientation(orient_robot[2])
neg_n_4_robot, neg_n_1_robot, neg_n_2_robot, neg_n_3_robot = n_4_robot+pi, n_1_robot+pi, n_2_robot+pi, n_3_robot+pi

In [74]:
local_c_robot = get_bounding_box_corners_local_coordinates(client_id, robot_information['object_handler'])
orientation_robot = robot_information['object_orientation']
orientation_robot.reverse() # notice that the orientation vector is reversed (theta is naturally the last, but we need to have it as the first value)
position_robot = robot_information['object_position']
global_coordinates_robot = map_local_coordinates_to_global_coordinates(local_c_robot, orientation_robot, position_robot)

In [76]:
global_coordinates_robot

[array([-1.87686982,  1.61772052, -0.0172811 ]),
 array([-1.87686957,  1.84318934, -0.0172845 ]),
 array([-2.17379736,  1.84318968, -0.01668138]),
 array([-2.17379762,  1.61772086, -0.01667798])]

In [75]:
global_coordinates_cuboid

[array([-1.47499979e+00,  5.00000359e-01,  5.96097668e-08]),
 array([-1.97499979e+00,  5.00000357e-01,  5.95995227e-08]),
 array([-1.97499978e+00,  3.56638395e-07,  5.95995227e-08]),
 array([-1.47499978e+00,  3.58617342e-07,  5.96097668e-08])]

In [81]:
normals_object = [n_1_obj, n_2_obj, n_3_obj, n_4_obj, n_1_obj]
normals_robot = [neg_n_4_robot, neg_n_1_robot, neg_n_2_robot, neg_n_3_robot, neg_n_4_robot]
lista_de_vertices_de_configuracao = []
for it_ext in range(0,len(normals_object)-1):
    print(" ")
    for it_int in range(0,len(normals_object)-1):
        answer_a = is_angle_between(normals_object[it_int], normals_robot[it_ext], normals_object[it_int+1])
        answer_b = is_angle_between(normals_robot[it_int], normals_object[it_ext],normals_robot[it_int+1])
        if answer_a or answer_b:
            print(f'colocar um vértice b{it_int+1}-a{it_ext}')
            if it_ext == 0:
                if it_int == 4:
                    lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[0] -global_coordinates_robot[-1])
                else:
                    lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[it_int+1] -global_coordinates_robot[-1])
            else:
                if it_int == 4:
                    lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[0] - global_coordinates_robot[it_ext-1])
                else:
                    lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[it_int+1] - global_coordinates_robot[it_ext-1])
        """
        elif answer_b:
            print(f'colocar um vértice b{it_int+1}-a{it_ext}')
            if it_ext == 0:
                lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[it_int+1] -global_coordinates_robot[-1])
            else:
                lista_de_vertices_de_configuracao.append(global_coordinates_cuboid[it_int+1] - global_coordinates_robot[it_ext-1])
        """

 
colocar um vértice b1-a0
colocar um vértice b3-a0
colocar um vértice b4-a0


IndexError: list index out of range

In [66]:
normals_object = ['v1_obj','v2_obj','v3_obj','v4_obj','v1_obj']
normals_robot = ['v4_robo','v1_robo','v2_robo','v3_robo','v4_robo']

for it_i in range(0,len(normals_object)-1):
    for it_j in range(0,len(normals_object)-1):
        pass
        #print(f'A CONTACT: is {normals_robot[it_i]} between {normals_object[it_j]} and {normals_object[it_j+1]}?')
        #print(f'B CONTACT: is {normals_object[it_i]} between {normals_robot[it_j]} and {normals_robot[it_j+1]}?')   
        #print(" ")     


In [67]:
normals_object = ['v1_obj','v2_obj','v3_obj','v4_obj','v1_obj']
normals_robot = ['v4_robo','v1_robo','v2_robo','v3_robo','v4_robo']
for it in range(0,len(normals_object)-1):
    print(f'A CONTACT: is {normals_robot[it]} between {normals_object[it]} and {normals_object[it+1]}?')
    print(f'B CONTACT: is {normals_object[it]} between {normals_robot[it]} and {normals_robot[it+1]}?')
    print(" ")

A CONTACT: is v4_robo between v1_obj and v2_obj?
B CONTACT: is v1_obj between v4_robo and v1_robo?
 
A CONTACT: is v1_robo between v2_obj and v3_obj?
B CONTACT: is v2_obj between v1_robo and v2_robo?
 
A CONTACT: is v2_robo between v3_obj and v4_obj?
B CONTACT: is v3_obj between v2_robo and v3_robo?
 
A CONTACT: is v3_robo between v4_obj and v1_obj?
B CONTACT: is v4_obj between v3_robo and v4_robo?
 


In [68]:
is_angle_between(0,0,1)

False